# ANSYS CFX live Job Monitoring

This Jupyter Notebook can be used to live monitor ANSYS CFX jobs on Rescale.  
The function `find_relevant_variables` below can be modified if other variables should be monitored. 

## Install required tools

In [ ]:
! pip install plotly ipywidgets==7.8.0

In [ ]:
import glob
import os
import subprocess
from pathlib import Path

import ipywidgets as widgets
import pandas as pd
import plotly.graph_objects as go
from IPython import display

## Helper functions

In [ ]:
def find_relevant_variables(mon_file):
    command = f'cfx5mondata -mon {mon_file} -showvars -varrule "CATEGORY = USER POINT"'
    cp = subprocess.run(
        command,
        shell=True,
        check=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True,
    )
    relevant_vars = [v for v in cp.stdout.splitlines() if v.startswith("USER POINT")]
    return relevant_vars


def get_csv_data(mon_file, relevant_variables, monitor_data_csv_file):
    command = f'cfx5mondata -mon {mon_file} -varlist "{";".join(relevant_variables)}" -out {monitor_data_csv_file}'
    subprocess.run(
        command,
        shell=True,
        check=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True,
    )
    df = pd.read_csv(monitor_data_csv_file)
    return df


def create_plot(mon_file):
    monitor_data_csv_file = mon_file.parent.joinpath(mon_file.stem + "_monitor.csv")
    relevant_variables = find_relevant_variables(mon_file)
    df = get_csv_data(mon_file, relevant_variables, monitor_data_csv_file)

    fig = go.Figure()
    for c in df.columns[1:]:
        fig.add_trace(
            go.Scatter(
                x=df[df.columns[0]], y=df[c], mode="lines+markers", name=df[c].name
            )
        )
    fig.update_layout(title=str(mon_file), showlegend=True)
    fig.update_xaxes(range=[0.0, None], title=df.columns[0])
    return fig

## Create Plot

In [ ]:
# start search in the home directory as there may be several work directories in a DOE
os.chdir(str(Path.home()))

mon_file_pattern = "**/mon"
mon_files = [Path(item) for item in glob.glob(mon_file_pattern, recursive=True)]

dropdown_mon = widgets.Dropdown(
    options=mon_files, values=mon_files[0], layout={"width": "max-content"}
)


def reload_button_on_click(obj):
    with output:
        fig = create_plot(dropdown_mon.value)
        display.clear_output(wait=True)
        display.display(fig)


reload_button = widgets.Button(description="Reload", icon="rotate-right")
reload_button.on_click(reload_button_on_click)

output = widgets.Output()


def dropdown_mon_eventhandler(change):
    with output:
        fig = create_plot(change["new"])
        display.clear_output(wait=True)
        display.display(fig)


dropdown_mon.observe(dropdown_mon_eventhandler, names="value")
display.display(dropdown_mon)
display.display(reload_button)

In [ ]:
display.display(output)

In [ ]:
dropdown_mon_eventhandler({"type": "change", "new": mon_files[0]})